In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [5]:
!pip install rasterio pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 38.4 MB/s eta 0:00:00


In [6]:
import rasterio as rio
import pandas as pd
import numpy as np

In [8]:
from numpy import meshgrid

def image_data_with_coords(filepath):

  with rio.open(filepath) as src:
    data = src.read()
    meta = src.meta

    height, width = data.shape[:2]

    print(meta)

    transform = meta["transform"]

    Y = np.arange(height).reshape(-1, 1)
    X, Y = meshgrid(np.arange(width), Y)
    x_coords, y_coords = rio.transform.xy(transform, X, Y)

#    x_coords, y_coords = rio.transform.xy(transform, np.arange(width).flatten(), np.arange(height).flatten())
    x_coords = [x if x is not None else np.nan for x in x_coords]
    y_coords = [y if y is not None else np.nan for y in y_coords]

    df = pd.DataFrame({
    "X": x_coords,
    "Y": y_coords,
    **{f"Band_{i+1}": data[i].flatten() for i in range(data.shape[0])}
    })

  return df

try:
  filepath = "/content/data/Images/sentinel2_2017-07-31.tiff"
  df = image_data_with_coords(filepath)
  if df is not None:
    print(df.head())
  else:
    print("Error: Invalid transformation matrix in image metadata")
except Exception as e:
  print(f"Error: {e}")



{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 1226, 'height': 1006, 'count': 13, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00011357131973898823, 0.0, -6.55216293,
       0.0, -8.988642147117354e-05, 37.7270859)}
Error: All arrays must be of the same length


In [9]:
def image_data_with_coords(filepath):

  with rio.open(filepath) as src:
    data = src.read()
    meta = src.meta

    height, width = data.shape[:2]

    print(meta)

    transform = meta["transform"]

    x_coords, y_coords = rio.transform.xy(transform, np.arange(width).flatten(), np.arange(height).flatten())
    x_coords = [x if x is not None else np.nan for x in x_coords]
    y_coords = [y if y is not None else np.nan for y in y_coords]

    df = pd.DataFrame({
    "X": x_coords,
    "Y": y_coords,
    **{f"Band_{i+1}": data[i].flatten() for i in range(data.shape[0])}
    })

  return df

try:
  filepath = "/content/data/Images/sentinel2_2017-07-31.tiff"
  df = image_data_with_coords(filepath)
  if df is not None:
    print(df.head())
  else:
    print("Error: Invalid transformation matrix in image metadata")
except Exception as e:
  print(f"Error: {e}")

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 1226, 'height': 1006, 'count': 13, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00011357131973898823, 0.0, -6.55216293,
       0.0, -8.988642147117354e-05, 37.7270859)}
Error: Input coordinates must be broadcastable to a 1d array


In [8]:
img = rio.open(r'/content/data/Images/sentinel2_2017-07-31.tiff')

array = img.read()
num_bands = array.shape[0]

height = img.shape[0]
width = img.shape[1]

cols, rows = np.meshgrid(np.arange(width), np.arange(height))
xs, ys = rio.transform.xy(img.transform, rows, cols)
x_coords = np.array(xs)
y_coords = np.array(ys)


array = np.concatenate((array, x_coords[None,:,:], y_coords[None,:,:]))

df = pd.DataFrame(array.reshape([num_bands+2,-1]).T, columns=[f"band_{i+1}" for i in range(num_bands)]+['x','y'])

column_names = df.columns

try:
  if df is not None:
    print(df)
    print(column_names)
    print(num_bands)
  else:
    print("Error: Invalid transformation matrix in image metadata")
except Exception as e:
  print(f"Error: {e}")


         band_1  band_2  band_3  band_4  band_5  band_6  band_7  band_8  \
0        0.0428  0.0403  0.0618  0.0685  0.1063  0.1593  0.1702  0.1747   
1        0.0428  0.0416  0.0643  0.0715  0.1063  0.1593  0.1702  0.1740   
2        0.0392  0.0441  0.0629  0.0721  0.1084  0.1618  0.1799  0.1830   
3        0.0392  0.0384  0.0595  0.0644  0.1084  0.1618  0.1799  0.1915   
4        0.0392  0.0442  0.0607  0.0697  0.1227  0.1602  0.1797  0.1863   
...         ...     ...     ...     ...     ...     ...     ...     ...   
1233351  0.0375  0.0414  0.0691  0.0805  0.1570  0.2139  0.2387  0.2376   
1233352  0.0375  0.0643  0.0926  0.1185  0.1570  0.2139  0.2387  0.2528   
1233353  0.0375  0.0469  0.0795  0.0937  0.1537  0.1933  0.2070  0.2227   
1233354  0.0375  0.0425  0.0685  0.0807  0.1537  0.1933  0.2070  0.2133   
1233355  0.0375  0.0484  0.0788  0.0947  0.1308  0.1812  0.1952  0.2181   

         band_9  band_10  band_11  band_12  band_13         x          y  
0        0.1873   0.2031

In [9]:
print(f"Length of x_coords: {len(x_coords)}")
print(f"Length of y_coords: {len(y_coords)}")


Length of x_coords: 1006
Length of y_coords: 1006


In [13]:
1226*1226


1503076

In [14]:
height*width

1233356

In [15]:
1006*1006

1012036

In [12]:
height*width

1233356